<a href="https://colab.research.google.com/github/Christy-cyber/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://ppa.launchpad.net/graph

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-04-08 00:22:32--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  1.22MB/s    in 0.8s    

2022-04-08 00:22:34 (1.22 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://<enter bucket name and rest of address here>/amazon_reviews_us_Luggage_v1_00.tsv"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Luggage_v1_00.tsv"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   40884699| R9CO86UUJCAW5|B00VGTN02Y|     786681372|Teenage Mutant Ni...|         Luggage|          3|            0|          0|   N|                Y|my review of this...|my review of this...| 2015-08-31|
|         US|   23208852|R3PR8X6QGVJ8B1|B005KIWL0E|     618251799|Kenneth Cole Reac...|         Luggage|          5|    

In [5]:
from pyspark.sql.functions import to_date
# Load in sql function to use columns
from pyspark.sql.functions import col
# Read in the Review dataset as a DataFrame--see above

In [6]:
# Get schema
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [7]:
# Filter total_votes column on >=20
total_votes_df = df.filter("total_votes>=20")
total_votes_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   20761040|R11IBSD5E6HPSD|B002B3FWXY|     677901073|Travelon Anti-The...|         Luggage|          1|           29|         31|   N|                Y|This bag was on m...|The strap broke!!...| 2015-08-31|
|         US|   12318409|R2KVWAYBPWK1OV|B011KEPZG8|     919734058|Iblue Canvas Leat...|         Luggage|          5|    

In [8]:
import pyspark.sql.functions as f
from pyspark.sql.types import IntegerType

# Create new column "results" to hold helpful_votes / total_votes using "withColumn" & create new dataframe
divided_votes_df = total_votes_df.withColumn("results", total_votes_df.helpful_votes / total_votes_df.total_votes)
divided_votes_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|           results|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+------------------+
|         US|   20761040|R11IBSD5E6HPSD|B002B3FWXY|     677901073|Travelon Anti-The...|         Luggage|          1|           29|         31|   N|                Y|This bag was on m...|The strap broke!!...| 2015-08-31|0.9354838709677419|
|         US|   12318409|R2KVWAYBPWK1OV|B011

In [9]:
# Filter "results" column for values >= 0.5 and create a new dataframe
filtered_votes_df = divided_votes_df.filter("results>=0.5")
filtered_votes_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|           results|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+------------------+
|         US|   20761040|R11IBSD5E6HPSD|B002B3FWXY|     677901073|Travelon Anti-The...|         Luggage|          1|           29|         31|   N|                Y|This bag was on m...|The strap broke!!...| 2015-08-31|0.9354838709677419|
|         US|   12318409|R2KVWAYBPWK1OV|B011

In [10]:
# Filter for reviews that were written as part of the vine program from the filtered_votes_df and create a new dataframe
vine_paid_df = filtered_votes_df.filter(filtered_votes_df["vine"] == "Y")
vine_paid_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|           results|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+------------------+
|         US|   15225359| RPJ7C4HP3BHXN|B00JR4A83O|     603403193|Fossil Men's Esta...|         Luggage|          5|           25|         29|   Y|                N|Beautiful Leather...|This Fossil bag i...| 2015-08-01|0.8620689655172413|
|         US|   53064080|R3SB08XK0M7993|B00S

In [11]:
# Filter for reviews that were not written as part of the vine program and create a new dataframe
vine_unpaid_df=filtered_votes_df.filter(filtered_votes_df["vine"] == "N")
vine_unpaid_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|           results|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+------------------+
|         US|   20761040|R11IBSD5E6HPSD|B002B3FWXY|     677901073|Travelon Anti-The...|         Luggage|          1|           29|         31|   N|                Y|This bag was on m...|The strap broke!!...| 2015-08-31|0.9354838709677419|
|         US|   12318409|R2KVWAYBPWK1OV|B011

In [12]:
# Determine total number of reviews for paid reviews
vine_paid_df.count()

21

In [13]:
# Filter for number of 5-star reviews for paid reviews
five_star_vine_paid_df = vine_paid_df.filter("star_rating = 5")
five_star_vine_paid_df.count()

10

In [14]:
# Determine percentage of 5-star reviews for paid reviews
percentage_paid_df = five_star_vine_paid_df.count() / vine_paid_df.count() * 100
percentage_paid_df

47.61904761904761

In [15]:
# Determine total number of reviews for unpaid reviews
vine_unpaid_df.count()

6690

In [16]:
# Filter for number of 5-star reviews for unpaid reviews
five_star_vine_unpaid_df = vine_unpaid_df.filter("star_rating = 5")
five_star_vine_unpaid_df.count()

3448

In [17]:
# Determine percentage of 5-star reviews for unpaid reviews
percentage_unpaid_df = five_star_vine_unpaid_df.count() / vine_unpaid_df.count() * 100
percentage_unpaid_df

51.53961136023917